# Registration Block-List
## Copay Gateway

This [web-app](https://enrollment.emgality.com/admin/block-warn) was created to add phone numbers and email addresses that should be blocked from registering for a copay card. Lilly offers incentives to patients that are a high financial risk to the company. The incentives allow patients to access their medication at faster rates.

![](https://intelligencepharma.files.wordpress.com/2019/06/simple-start.png?w=350&h=200&crop=1)

It was noted that the expected registration flow for the emgality program was being exploited by pharmacies. Pharmacies were downloading and activating the card on behalf of the patient, which is against the rules!

The username for the [web-app](https://enrollment.emgality.com/admin/block-warn) is 'Admin', reach out to me for the password if you ever want to log in. 

## Log in to PCP

In [23]:
# abstracted utility functions are stored in a different file
import pcp_utility as pcp

# connecting to prod
sf = pcp.connect_to_prod(True)
print(sf)

# Registration Data
"Registration" is the data object in which we are storing information that is passed in from the [savings and support gateway](https://enrollment.emgality.com/copay/emgality/contact-info).

## Data caching
I am going to pull all historical registration data. From there I am going to save the data as a JSON file so that I do not need to query salesforce every time i update the block list. I can just query new registrations for the day and add them to the JSON structure.

In [29]:
"""
Yes, this design is garbage, build a django app to depreciate existing automation between the copay gateway and the PCP. The gateway code can be found here: https://github.com/EliLillyCo/lusa-enrollment-services 

It is exclusively javascript. Building a Django app allows us to build the Javascript ontop of a layer of python code. The python code will efficiently process the data and perform procedures, such as blocking an email/phone address.
"""

# we only need a few fields for determining if a phone or email should be added
soql = "SELECT Name, Email__c, Phone__c, Createddate, DTPC_First_Name__c, DTPC_Last_Name__c, DTPC_Program_Registration__c FROM DTPC_Registration__c WHERE Registration_Type__c = 'Copay / Saving Card'"

registrations = pcp.query(sf, soql)

with open('support_structures/registration_blocking.json', 'w') as fp:
    json.dump(registrations, fp)

In [31]:
# load from json
import json

with open('support_structures/registration_blocking.json', 'r') as fp:
    registrations = json.load(fp)

print(len(registrations))

345956


## Parsing and cleaning data
We are going to look at each email and phone address that is collected from the registrations and try to determine whether or not we should block it.

To do this we will create a dictionary with the phone as the key and a dictionary with the email as a key. The value for each key will be the registration data.

We look at each registration record and add it into the email and phone dictionaries. If a specific email address or phone number is already in the map, the registration will be added to a list.

In [34]:
def create_maps(registrations):
    email_map = {}
    phone_map = {}
    for registration in registrations:
        email = registration['Email__c']
        phone = registration['Phone__c']
        if email in email_map.keys():
            email_map[email].append(registration)
        else:
            email_map[email] = [registration]
        if phone in phone_map.keys():
            phone_map[phone].append(registration)
        else:
            phone_map[phone] = [registration]
    # return both maps
    return [email_map, phone_map]

# setting a value to use for formatted data
registration_maps = create_maps(registrations)

## Defining the block list
This chunk of code will determine which phones and emails need to be blocked. The first function will create the blocklist and it calls a support function that analyzes the names associated with each email and phone number.

The data was formatted in the last function. These two functions will parse through the formatted data and determine which emails and phones need to be blocked from registrering.

In [39]:
# importing packages
import itertools
from fuzzywuzzy import fuzz

# sets up the block list by analyzing names associated with each email/phone
def create_block_list(maps):
    """ """
    email_map = maps[0]
    phone_map = maps[1]
    
    block_email = {}
    block_phone = {}
    
    for email in email_map:
        names = []
        for registration in email_map[email]:
            name = registration['DTPC_First_Name__c'] + registration['DTPC_Last_Name__c']
            names.append(name.lower())

        if len(set(names)) > 1:
            block_email[email] = set(names)

    
    for phone in phone_map:
        names = []
        for registration in phone_map[phone]:
            name = registration['DTPC_First_Name__c'] + registration['DTPC_Last_Name__c']
            names.append(name.lower())
        if len(set(names)) > 1:
            block_phone[phone] = set(names)
    
    # we call the function below (both block lists need the same cleaining)
    blocked_emails = clean_block_list(block_email)
    blocked_phones = clean_block_list(block_phone)

    # our emails and phones that need to be blocked!
    return [blocked_emails, blocked_phones]


def clean_block_list(block_list):
    clean_block_list = {}
    # we are able to add exceptions to this list so that these are not blocked
    email_exceptions = ['david.reed@lilly.com', 'umurthy@lilly.com', 'birge_monty_donovan@network.lilly.com',
                        'mackert_preston_reed@lilly.com', 'fields_megan_christine@lilly.com',
                        'telford_dylan_colin@lilly.com', 'harris_akiya_charene@lilly.com']
    phone_exceptions = ['219-455-7477', '317-626-9286', '317-374-8025']
    for entry in block_list:
        if entry not in email_exceptions or entry not in phone_exceptions:
            name_list = block_list[entry]
            ratios = []
            for name1, name2 in itertools.combinations(name_list, 2):
                ratio = fuzz.ratio(name1, name2)
                ratios.append(ratio)
            min_ratio = ratios[0]
            for ratio in ratios:
                if ratio < min_ratio:
                    min_ratio = ratio
            if min_ratio < 80:
                clean_block_list[entry] = block_list[entry]

    return clean_block_list


c:\Users\c266139\Documents\web_services\integration_engine\django-salesforce\env\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [47]:
# setting the variables for the block lists
block_lists = create_block_list(registration_maps)
blocked_emails = block_lists[0].keys()
blocked_phones = block_lists[1].keys()

print(len(blocked_emails))

4879


## I/O functions 
Gives us the ability to read and write files that maintain our block lists. We only want to add new phone numbers and email addresses to our block list every time we update it. For this reason, we are taking the set value of each newly formed block list and subtracting the set of all logged values.

The result is a single list of only the emails and phones that were found in the new query and are not logged on the web app.

In [49]:
def read_master_list(filename):
    file = open(filename, 'r')
    emails = file.read().splitlines()
    file.close()
    return emails


def write_master(filename, block_list):
    with open(filename, 'w') as file:
        for entry in block_list:
            file.write("%s\n" % entry)

In [50]:
email_list = read_master_list('support_structures/block_lists/email_list.txt')
phone_list = read_master_list('support_structures/block_lists/phone_list.txt')

new_emails = set(blocked_emails) - set(email_list)
new_phones = set(blocked_phones) - set(phone_list)

print(len(new_emails))
print(len(new_phones))

221
236


# Accessing the web application
These functions will actually access the web app that can be found at: https://enrollment.emgality.com/admin/block-warn/emails and https://enrollment.emgality.com/admin/block-warn/phone-numbers

We will define the functions in these code chunks and then call them to analyze our new phones and emails that need to be blocked.

In [57]:
# we will use web-based magic to access the website that we built
import requests
import time
import random

# setting up the functions
def block_emails_on_portal(emails_to_block, proxies):
    proxies = {
        'http': 'http://foo:bar@us_proxy_indy.xh1.lilly.com:9000',
        'https': 'http://foo:bar@us_proxy_indy.xh1.lilly.com:9000'
    }

    # Fill in your details here to be posted to the login form.
    payload = {
        'username': os.environ.get('BLOCK_USER'),
        'password': os.environ.get('BLOCK_PASSWORD')
    }

    if proxies:
        with requests.Session() as session:
            login_response = session.post('https://enrollment.emgality.com/api/v2/admin/login', data=payload,proxies=proxies)
            
            response_dict = eval(login_response.text)
            cookies = {'admin': response_dict['token']}

            headers = {
                'referer': "https://enrollment.emgality.com/admin/block-warn/emails", 'user_agent': "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36"
            }

            for email in emails_to_block:
                time.sleep(random.randrange(0, 5))
                params = {'email': email, 'level': "block"}
                r = session.post('https://enrollment.emgality.com/api/v2/admin/block-warn/emails', data=params, headers=headers, proxies=proxies, cookies=cookies)
    
    else:
        with requests.Session() as session:
            login_response = session.post('https://enrollment.emgality.com/api/v2/admin/login', data=payload)
            
            response_dict = eval(login_response.text)
            cookies = {'admin': response_dict['token']}

            headers = {
                'referer': "https://enrollment.emgality.com/admin/block-warn/emails", 'user_agent': "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36"
            }

            for email in emails_to_block:
                time.sleep(random.randrange(0, 5))
                params = {'email': email, 'level': "block"}
                r = session.post('https://enrollment.emgality.com/api/v2/admin/block-warn/emails', data=params, headers=headers, cookies=cookies)

In [45]:
def block_phones_on_portal(numbers_to_block, proxies):
    proxies = {
        'http': 'http://foo:bar@us_proxy_indy.xh1.lilly.com:9000',
        'https': 'http://foo:bar@us_proxy_indy.xh1.lilly.com:9000'
    }

    # Fill in your details here to be posted to the login form.
    payload = {
        'username': os.environ.get('BLOCK_USER'),
        'password': os.environ.get('BLOCK_PASSWORD')
    }

    if proxies:
        # Use 'with' to ensure the session context is closed after use.
        with requests.Session() as session:
            login_response = session.post('https://enrollment.emgality.com/api/v2/admin/login', data=payload, proxies=proxies)
            
            response_dict = eval(login_response.text)
            cookies = {'admin': response_dict['token']}

            headers = {
                'referer': "https://enrollment.emgality.com/admin/block-warn/emails", 'user_agent': "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36"
            }

            for phone_number in numbers_to_block:
                time.sleep(random.randrange(0, 5))
                params = {'phoneNumber': phone_number, 'level': "block"}
                r = session.post('https://enrollment.emgality.com/api/v2/admin/block-warn/phone-numbers', data=params, headers=headers, proxies=proxies, cookies=cookies)
    else:
        # Use 'with' to ensure the session context is closed after use.
        with requests.Session() as session:
            login_response = session.post('https://enrollment.emgality.com/api/v2/admin/login', data=payload)
            
            response_dict = eval(login_response.text)
            cookies = {'admin': response_dict['token']}

            headers = {
                'referer': "https://enrollment.emgality.com/admin/block-warn/emails", 'user_agent': "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36"
            }

            for phone_number in numbers_to_block:
                time.sleep(random.randrange(0, 5))
                params = {'phoneNumber': phone_number, 'level': "block"}
                r = session.post('https://enrollment.emgality.com/api/v2/admin/block-warn/phone-numbers', data=params, headers=headers, cookies=cookies)

# Adding our new emails/phones to the LIST!
This will call our functions to add all of the new emails and phones to the block list.

In [61]:
if len(new_emails) > 0:
    print("accessing web portal and updating the email block list...")
    block_emails_on_portal(new_emails, True)

if len(new_phones) > 0:
    print("accessing web portal and updating the phone block list...")
    block_phones_on_portal(new_phones, True)

write_master('support_structures/block_lists/email_list.txt', blocked_emails)
write_master('support_structures/block_lists/phone_list.txt', blocked_phones)

accessing web portal and updating the email block list...
accessing web portal and updating the phone block list...
